# Data Loading

In [7]:
import os
os.chdir('electrochemistry_sean_mcintosh/electrochem')

In [8]:
!ls

__pycache__			     transform_excel_files_to_csv.ipynb
data.py				     utils.py
data_loading.ipynb		     wandb
data_plots.ipynb		     waylon_constants.py
paperspace_setup.sh		     waylon_generic_utils.py
train_attention.ipynb		     waylon_keras_utils.py
train_lstm.ipynb		     waylon_layer_utils.py
transform_csvs_to_keras_input.ipynb


In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
from pathlib import Path
import seaborn as sns
from tqdm.notebook import trange, tqdm
import wandb

sns.set()

DATA_DIR = Path('../data')

# Do not read in 'Mix 1' sheet, as that has been updated in 'mix_1_updated.xlsx'
sheet_names = ['Seawater - No Heavy Metals', 'Copper', 'Cadmium', 'Lead']
xcel = pd.read_excel(DATA_DIR / 'main.xlsx', sheet_name=sheet_names)
# Read in updated mix sheet
mix = pd.read_excel(DATA_DIR / 'mix_1_updated.xlsx')

seawater = xcel['Seawater - No Heavy Metals']
copper = xcel['Copper']
cadmium = xcel['Cadmium']
lead = xcel['Lead']

In [ ]:
seawater['label'] = 'Sw'
seawater = seawater.drop(['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2'], axis=1)

In [ ]:
seawater.head(3)

,1,0.996,0.992,0.988,0.984,0.98,0.976,0.972,0.968,0.964,...,0.968.1,0.972.1,0.976.1,0.98.1,0.984.1,0.988.1,0.992.1,0.996.1,1.1,label
0,-0.693472,-0.674730,-0.718462,-0.649740,-0.693472,-0.637245,-0.674730,-0.624750,-0.662235,-0.618502,...,4.410735,4.454467,4.573170,4.610655,4.748100,4.779337,4.923030,4.966762,5.110455,Sw
1,-0.312375,-0.274890,-0.306127,-0.256147,-0.281137,-0.249900,-0.274890,-0.237405,-0.274890,-0.206167,...,3.461115,3.561075,3.642292,3.767242,3.842212,3.992152,4.060875,4.223310,4.298280,Sw
2,-0.574770,-0.637245,-0.562275,-0.606007,-0.531037,-0.581017,-0.506047,-0.556027,-0.487305,-0.537285,...,3.854707,3.985905,4.092112,4.198320,4.304527,4.435725,4.573170,4.704367,4.860555,Sw


In [ ]:
copper['label'] = 'Cu'
cadmium['label'] = 'Cd'
lead['label'] = 'Pb'

In [ ]:
lead.columns

Index([   'Unnamed: 0',       'Analyte', 'Concentration',               1,
                 0.996,           0.992,           0.988,           0.984,
                  0.98,           0.976,
       ...
             '0.968.1',       '0.972.1',       '0.976.1',        '0.98.1',
             '0.984.1',       '0.988.1',       '0.992.1',       '0.996.1',
                 '1.1',         'label'],
      dtype='object', length=1006)

In [ ]:
copper = copper.drop(['Unnamed: 0', 'Unnamed: 1', 'Concentration'], axis=1)
cadmium = cadmium.drop(['Unnamed: 0', 'Analyte', 'Concentration'], axis=1)
lead = lead.drop(['Unnamed: 0', 'Analyte', 'Concentration'], axis=1)

In [ ]:
dfs = [copper, cadmium, lead, seawater]
for df in dfs:
    print(df.shape)

(23, 1003)
(36, 1003)
(36, 1003)
(80, 1003)


In [ ]:
df = pd.concat(dfs, ignore_index=True)

In [ ]:
df.label.value_counts(normalize=True)

Sw    0.457143
Pb    0.205714
Cd    0.205714
Cu    0.131429
Name: label, dtype: float64

In [ ]:
df.head()

,1,0.996,0.992,0.988,0.984,0.98,0.976,0.972,0.968,0.964,...,0.968.1,0.972.1,0.976.1,0.98.1,0.984.1,0.988.1,0.992.1,0.996.1,1.1,label
0,-3.225950,-3.350025,-3.350025,-3.350025,-3.350025,-3.287988,-3.350025,-3.287988,-3.350025,-3.287988,...,8.809325,8.933400,8.995438,9.119513,9.243588,9.367663,9.491738,9.615813,9.739888,Cu
1,-4.032438,-4.094475,-4.094475,-4.094475,-4.094475,-4.094475,-4.094475,-4.032438,-4.032438,-4.156513,...,5.955600,6.079675,6.203750,6.265788,6.389863,6.451900,6.700050,6.700050,6.886163,Cu
2,-0.496300,-0.558338,-0.558338,-0.496300,-0.558338,-0.496300,-0.558338,-0.434263,-0.558338,-0.496300,...,3.536138,3.536138,3.660213,3.722250,3.846325,3.846325,4.094475,4.156513,4.280588,Cu
3,-0.806488,-0.744450,-0.744450,-0.744450,-0.744450,-0.682413,-0.682413,-0.682413,-0.620375,-0.682413,...,3.784288,3.846325,3.970400,4.032438,4.156513,4.280588,4.404663,4.404663,4.590775,Cu
4,-0.434263,-0.496300,-0.434263,-0.434263,-0.434263,-0.372225,-0.434263,-0.372225,-0.434263,-0.434263,...,2.791688,2.853725,2.853725,2.977800,3.101875,3.163913,3.225950,3.287988,3.412063,Cu


In [ ]:
df.to_csv(DATA_DIR / 'four_class_dataset.csv')

In [ ]:
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

In [ ]:
df.shape

(175, 1003)